<a href="https://colab.research.google.com/github/thiagokato80/alura_projects/blob/main/Alura_Projetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando bibliotecas necessárias
!pip install -q -U google-generativeai
!pip install bs4
!pip install langchain
!pip install langchain_google_genai
!pip install sentence-transformers
!pip install chromadb

In [2]:
# Importar bibliotecas necessárias

import requests
import os
from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import google.generativeai as genai
import google.ai.generativelanguage as glm
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain_core.messages import HumanMessage

from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import warnings
warnings.filterwarnings("ignore")

In [3]:
genai.configure()

In [4]:
# Cria um dicionário vazio para armazenar as informações do site
dicionario = {}
texto = []
texto_final = ""
doc_final = ""
question = ""

# prompt: criar um dicionário com as informações contidas em um site
# A ideia era subir PDFs também, etc, mas não tive tempo de ajustar o código (no meu GIT há um exemplo com PDF)
urls = ("https://www.jota.info/eleicoes/quem-sao-os-possiveis-candidatos-a-prefeito-de-sao-paulo-nas-eleicoes-municipais-de-2024-25042024",
        "https://www.cnnbrasil.com.br/politica/boulos-nunes-tabata-datena-quem-os-partidos-querem-colocar-na-disputa-pela-prefeitura-de-sp/",
        "https://www.cnnbrasil.com.br/blogs/pedro-venceslau/eleicoes/boulos-nunes-e-tabata-turbinam-planos-de-governo-e-buscam-nomes-ecumenicos-saiba-quem-sao-os-coordenadores/",
        )
# insira mais URLs separando por vírgula

# Define a URL do site que você deseja extrair informações
for site in urls:
  url = site

  loader = WebBaseLoader(url)
  docs = loader.load()
  pagina = docs[0].page_content
  texto.append(pagina)

for valor in texto:
  texto_final += valor + " "

splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=[
                    "\n\n",
                    "\n",
                    " ",
                    ".",
                    ",",
                    "\u200b",  # Zero-width space
                    "\uff0c",  # Fullwidth comma
                    "\u3001",  # Ideographic comma
                    "\uff0e",  # Fullwidth full stop
                    "\u3002",  # Ideographic full stop
                    "",
                ]
    )
splitDocs = splitter.split_text(texto_final)

for valor in splitDocs:
  doc_final += valor + " "

# Converte o texto para formato 'Document' do LangChain
docs = [Document(page_content=doc_final, metadata={"source": "local"})]


# Adiciona ao dicionário  as informações extraídas
dicionario = docs

In [7]:
# Gemini Embeddings - Apagar pasta chroma_db se for rodar com o Embedding diferente deste

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                                 google_api_key="sua_chave_api_google"
                                                 )

In [8]:
# Cria uma base vetorial com as informações dos sites
vectorstore = Chroma.from_documents(
                     documents=docs,                 # Data
                     embedding=gemini_embeddings,   # Embedding model
                     persist_directory="./chroma_db" # Directory to save data
                     )

In [9]:
# Carrega as base vetorial guardada
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=gemini_embeddings   # Embedding model
                   )

In [10]:
# LLM
llmchat = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",
                            temperature=0.7, top_p=0.85,
                            google_api_key="sua_chave_api_google"
                             )

In [12]:
###### NÃO ENTENDI O MOTIVO DE TER QUE RODAR UMA SEGUNDA VEZ ESTA CÉLULA PARA DAR AS RESPOSTAS
###### A PRIMEIRA ELE NÃO BUSCA NOS ARQUIVOS, JÁ NA SEGUNDA VEZ ELE CONSEGUE RECUPERAR AS INFORMAÇÕES SALVAS
chat_history = []

# Perguntas e Respostas
while True:
    prompt = input("\nEsperando prompt: ")
    if prompt == "fim":
        break
    if prompt.strip() == "":
        continue

    retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 1})

    # Prompt template contextualizado
    contextualize_q_system_prompt = """Dado o histórico da conversa, as informações recuperadas, e a última pergunta, formule uma nova pergunta que possa ser utilizada sem perder o histórico da conversa.
    Somente reformule-a caso necessário, caso contrário deixe a como está."""

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
            )

    # Prompt Template da Pergunta
    qa_system_prompt = """Você é um assistente de perguntas e respostas.
    Utilize as informações recuperadas sobre os candidatos nas eleições de 2024 para a prefeitura da cidade de São Paulo e responda a pergunta contextualizada.
    Se não souber responder diga apenas que não sabe a resposta.
    Não forneça como resposta a pergunta de contexto.
    Agradeça ao final de cada resposta.\n
    Pergunta: {question} \nContexto: {context} \nResposta:
    """

    qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
        )

    question_answer_chain = create_stuff_documents_chain(llmchat, qa_prompt)

    history_aware_retriever = create_history_aware_retriever(
            llmchat, retriever,
            qa_prompt
            )

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    question = prompt
    res = rag_chain.invoke({"input": question, "chat_history": chat_history, "question" : qa_prompt, "context" : retriever})
    chat_history.extend([HumanMessage(content=question), res["answer"]])
    print("\n\n> Question:")
    print(question)
    print(res['answer'])


Esperando prompt: De acordo com o Datafolha, qual o percentual de intenção de votos para Boulos?


> Question:
De acordo com o Datafolha, qual o percentual de intenção de votos para Boulos?
    De acordo com o Datafolha, Boulos tem 32% das intenções de voto. Agradeço a sua pergunta. 


Esperando prompt: E para Nunes?


> Question:
E para Nunes?
    Nunes tem 24% das intenções de voto, segundo o Datafolha. Agradeço a sua pergunta. 


Esperando prompt: fim


In [13]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for i, message in enumerate(chat_history):
  display(to_markdown(f'**{chat_history[i]}**'))
  print('-------------------------------------------')

> **content='De acordo com o Datafolha, qual o percentual de intenção de votos para Boulos?'**

-------------------------------------------


> **    De acordo com o Datafolha, Boulos tem 32% das intenções de voto. Agradeço a sua pergunta. 
> **

-------------------------------------------


> **content='E para Nunes?'**

-------------------------------------------


> **    Nunes tem 24% das intenções de voto, segundo o Datafolha. Agradeço a sua pergunta. 
> **

-------------------------------------------
